# Hand Gesture Recognition using SVM
This notebook will:
1. Download a hand gesture dataset from Kaggle
2. Extract hand landmarks using MediaPipe
3. Train an SVM classifier to recognize gestures

In [ ]:
# Install dependencies (only run once)
!pip install kaggle mediapipe opencv-python scikit-learn matplotlib seaborn tqdm

## Upload your Kaggle API key
Download it from https://www.kaggle.com/account (kaggle.json)

In [ ]:
from google.colab import files  # If using Colab
files.upload()  # Upload kaggle.json

In [ ]:
import os
import zipfile

# Move kaggle.json to correct location
os.makedirs('/root/.kaggle', exist_ok=True)
!mv kaggle.json /root/.kaggle/kaggle.json
!chmod 600 /root/.kaggle/kaggle.json

# Download dataset
!kaggle datasets download -d lmzintgraf/hand-gesture-recognition-dataset

# Unzip dataset
with zipfile.ZipFile('hand-gesture-recognition-dataset.zip', 'r') as zip_ref:
    zip_ref.extractall('data')

In [ ]:
# Imports
import cv2
import numpy as np
import mediapipe as mp
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [ ]:
# Extract keypoints from hand image
mp_hands = mp.solutions.hands

def extract_keypoints(image_path):
    img = cv2.cvtColor(cv2.imread(image_path), cv2.COLOR_BGR2RGB)
    with mp_hands.Hands(static_image_mode=True, max_num_hands=1) as hands:
        res = hands.process(img)
        if res.multi_hand_landmarks:
            lm = res.multi_hand_landmarks[0].landmark
            return np.array([[p.x, p.y, p.z] for p in lm]).flatten()
    return None

In [ ]:
# Load all images and extract features
def load_dataset(data_dir):
    X, y, labels = [], [], sorted(os.listdir(data_dir))
    for idx, label in enumerate(labels):
        folder = os.path.join(data_dir, label)
        for img in tqdm(os.listdir(folder), f"Loading {label}"):
            pts = extract_keypoints(os.path.join(folder, img))
            if pts is not None:
                X.append(pts)
                y.append(idx)
    return np.array(X), np.array(y), labels

X, y, class_names = load_dataset('data')

In [ ]:
# Train/test split and SVM training
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y)
scaler = StandardScaler().fit(X_train)
X_train_s, X_test_s = scaler.transform(X_train), scaler.transform(X_test)

clf = SVC(kernel='rbf', C=10).fit(X_train_s, y_train)
y_pred = clf.predict(X_test_s)

print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred, target_names=class_names))

In [ ]:
# Confusion matrix
cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(6,5))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=class_names, yticklabels=class_names)
plt.xlabel("Predicted"); plt.ylabel("True"); plt.title("Confusion Matrix")
plt.show()